In [23]:
import sys
import numpy as np
import tracemalloc
import matplotlib.pyplot as plt
sys.path.append('C:/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/QiskitProjects/VariationalWavefunction/')
from quantum_kernel.code.feature_maps import IQP
from quantum_kernel.code.feature_maps import Heisenberg
from quantum_kernel.code.utils import get_quantum_kernel,get_projected_quantum_kernel
from quantum_kernel.code.quantum_kernels import projected_kernels
from quantum_kernel.code.utils import get_dataset
from qiskit.circuit import Parameter,ParameterVector
from qiskit.circuit.library.data_preparation.pauli_feature_map import PauliFeatureMap
from qiskit import QuantumCircuit
from qiskit import Aer
import qiskit.quantum_info as qi
from sklearn.svm import SVC

In [70]:
num_qubits=6
#test=IQP.Sparse_IQP(num_qubits,2,"full",5,int_time_scale=1)
test=Heisenberg.Heisenberg1DFeatureMap(num_qubits,10,2,'Basis_random',0)
x_train, x_test, y_train, y_test = get_dataset('plasticc', num_qubits, 800, 200)

Test to make sure that changing the batch size does not change the kernel matrix.

In [71]:
small_batch_test_kernel=get_projected_quantum_kernel(test,simulation_method='statevector',batch_size=25,gamma=0.5)
large_batch_test_kernel=get_projected_quantum_kernel(test,simulation_method='statevector',batch_size=10000,gamma=0.5)

In [72]:
sb_kernel_matrix=small_batch_test_kernel.evaluate(x_vec=x_train[0:800],y_vec=x_train[0:800])
lb_kernel_matrix,_=large_batch_test_kernel.evaluate(x_vec=x_train[0:800],y_vec=x_train[0:800])

assert(np.all(sb_kernel_matrix==lb_kernel_matrix)),'Changing batch size changes kernel matrix'

Test to make sure that we can change the measured qubits.

In [21]:
mq=[[i] for i in range(num_qubits)]
large_batch_test_kernel.set_measured_qubits(mq)
kernel_matrix2=large_batch_test_kernel.evaluate(x_vec=x_train[0:100],y_vec=x_train[0:100])

In [ ]:
mq=[[i,i+1] for i in range(num_qubits-1)]
large_batch_test_kernel.set_measured_qubits(mq)
kernel_matrix3=large_batch_test_kernel.evaluate(x_vec=x_train[0:100],y_vec=x_train[0:100])

top stats
[<Statistic traceback=<Traceback <Frame filename='/home/lslattery/anaconda3/envs/quantum_kernel/lib/python3.9/copy.py' lineno=279>> size=100522448 count=1152801>, <Statistic traceback=<Traceback <Frame filename='/home/lslattery/anaconda3/envs/quantum_kernel/lib/python3.9/site-packages/qiskit/circuit/quantumcircuit.py' lineno=2116>> size=74201600 count=1510300>, <Statistic traceback=<Traceback <Frame filename='/home/lslattery/anaconda3/envs/quantum_kernel/lib/python3.9/site-packages/qiskit/circuit/parameterexpression.py' lineno=49>> size=29968704 count=138744>]


Check that the kernel matrix is positive semidefinite,symmetric.

In [ ]:
assert(np.all(np.real(np.linalg.eigvals(sb_kernel_matrix)) > -1e-13)),'Not PSD'
assert(np.all(np.real(np.linalg.eigvals(lb_kernel_matrix)) > -1e-13)),'Not PSD'
assert(np.all(np.real(np.linalg.eigvals(kernel_matrix2)) > -1e-1)),'Not PSD'
assert(np.all(np.real(np.linalg.eigvals(kernel_matrix3)) > -1e-13)),'Not PSD'

In [ ]:
assert(np.allclose(sb_kernel_matrix-sb_kernel_matrix.T,0)),'Not symmetric'
assert(np.allclose(lb_kernel_matrix-lb_kernel_matrix.T,0)),'Not symmetric'
assert(np.allclose(kernel_matrix2-kernel_matrix2.T,0)),'Not symmetric'
assert(np.allclose(kernel_matrix3-kernel_matrix3.T,0)),'Not symmetric'

In [ ]:
#assert(np.isclose(np.trace(sb_kernel_matrix),sb_kernel_matrix.shape[0])),'Trace(K) != N'
#assert(np.isclose(np.trace(lb_kernel_matrix),lb_kernel_matrix.shape[0])),'Trace(K) != N'
#assert(np.isclose(np.trace(kernel_matrix2),kernel_matrix2.shape[0])),'Trace(K) != N'
#assert(np.isclose(np.trace(kernel_matrix3),kernel_matrix3.shape[0])),'Trace(K) != N'